# **Feature Extraction Methods: Imbalanced Data Without Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random
import IPython.display as ipd
from functools import partial

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(not_annotated_splt)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

In [15]:
train_data['species'].value_counts()

species
Alcedo atthis                39
Himantopus himantopus        36
Porphyrio porphyrio          33
Acrocephalus melanopogon     32
Gallinula chloropus          32
Fulica atra                  28
Tachybaptus ruficollis       27
Botaurus stellaris           27
Charadrius alexandrinus      26
Ardea purpurea               26
Dendrocopos minor            25
Motacilla flava              24
Acrocephalus arundinaceus    23
Ciconia ciconia              22
Ixobrychus minutus           21
Acrocephalus scirpaceus      20
Circus aeruginosus           19
Coracias garrulus            11
Anas platyrhynchos            9
Anas strepera                 5
Name: count, dtype: int64

# **Creating a class to do the extraction**

In [6]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=60, n_mfcc=20, n_chroma=12, features=['mfcc'], normalize=True, avgpool=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'rms', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      maxpool (bool): Whether to maxpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'rms', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'rms', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features = self.feature_extraction(self.val_df, window_size=self.window_size)

    # process the features by average pooling
    self.train_features, self.val_features = self.process_features(self.train_features, self.val_features)

    
  

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def bandpass_filter(self, audio, lowcut=800, highcut=8000, order=4):
    nyquist = 0.5 * self.sr  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist

    b, a = butter(order, [low, high], btype='band')
    filtered_audio = filtfilt(b, a, audio)
    return filtered_audio
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_rms(self, window):
    return librosa.feature.rms(y=window)

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels)
    if self.normalize:
      return librosa.util.normalize(mel)
    else:
      return mel
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    for each in train_features_dict.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'rms':
        n_features=1
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_features_dict[each]
      val_feature = val_features_dict[each]

      if self.avgpool:
        train_features_dict[each], val_features_dict[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_features_dict[each], val_features_dict[each] = train_features_dict[each], val_features_dict[each]
    
    return train_features_dict, val_features_dict
      

  def feature_extraction(self, dataframe, window_size, filter=True):
    y = [] # To hold the labels
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          audio = self.normalize_audio(audio)

          audio = self.pad_with_noise(audio, window_length=self.window_size, window_samples=len(audio))
          # print(len(sample))

          if filter:
                  audio = self.bandpass_filter(audio)

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(audio, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)

          # If not using average pooling, return resized features
    return y, features_dict

# **No Average Pooling**

## **Window Size = 6s**

### **['melspectrogram']**

In [16]:
features_list = ['melspectrogram']

In [17]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:03<00:00,  7.63it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:16<00:00,  8.12it/s]


In [18]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60, 259)

In [19]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [20]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60, 259)

### Encode Classes

In [21]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [22]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [23]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [24]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           2.09090982e-14, 1.00035500e-07, 4.10201886e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           2.42591675e-14, 1.00793242e-07, 4.13385302e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.06456201e-14, 1.02097112e-07, 4.18868937e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.76671411e-08, 2.59462464e-08, 1.32848568e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           2.24950763e-10, 2.39690650e-09, 9.34205219e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           6.16144049e-13, 1.95562647e-09, 8.06973052e-08]],
  
         [[1.73426295e-05, 3.63408438e-06, 4.80290598e-13, ...,
           1.60876280e-14, 7.00188949e-08, 1.07775013e-06],
          [1.84753151e-05, 3.87795431e-06, 5.39064175e-13, ...,
           2.48021127e

### Save the merged dictionary to a pkl

In [25]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_6s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [26]:
features_list = ['melspectrogram', 'mfcc']

In [27]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [03:27<00:00,  2.34it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:31<00:00,  4.13it/s]


In [28]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60, 259)

'mfcc'

(7105, 20, 259)

In [29]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60, 259)

'mfcc'

(1896, 20, 259)

### Encode Classes

In [30]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [31]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [32]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [33]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           2.09090982e-14, 1.00035500e-07, 4.10201886e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           2.42591675e-14, 1.00793242e-07, 4.13385302e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.06456201e-14, 1.02097112e-07, 4.18868937e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.76671411e-08, 2.59462464e-08, 1.32848568e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           2.24950763e-10, 2.39690650e-09, 9.34205219e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           6.16144049e-13, 1.95562647e-09, 8.06973052e-08]],
  
         [[1.73426295e-05, 3.63408438e-06, 4.80290598e-13, ...,
           1.60876280e-14, 7.00188949e-08, 1.07775013e-06],
          [1.84753151e-05, 3.87795431e-06, 5.39064175e-13, ...,
           2.48021127e

### Save the merged dictionary to a pkl

In [34]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_6s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [35]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [36]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [05:24<00:00,  1.49it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:58<00:00,  2.23it/s]


In [37]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60, 259)

'mfcc'

(7105, 20, 259)

'chroma'

(7105, 12, 259)

In [38]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60, 259)

'mfcc'

(1896, 20, 259)

'chroma'

(1896, 12, 259)

### Encode Classes

In [39]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [40]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [41]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [42]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           2.09090982e-14, 1.00035500e-07, 4.10201886e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           2.42591675e-14, 1.00793242e-07, 4.13385302e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.06456201e-14, 1.02097112e-07, 4.18868937e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.76671411e-08, 2.59462464e-08, 1.32848568e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           2.24950763e-10, 2.39690650e-09, 9.34205219e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           6.16144049e-13, 1.95562647e-09, 8.06973052e-08]],
  
         [[1.73426295e-05, 3.63408438e-06, 4.80290598e-13, ...,
           1.60876280e-14, 7.00188949e-08, 1.07775013e-06],
          [1.84753151e-05, 3.87795431e-06, 5.39064175e-13, ...,
           2.48021127e

### Save the merged dictionary to a pkl

In [43]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_6s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [44]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [45]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [06:06<00:00,  1.32it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:15<00:00,  1.72it/s]


In [46]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(7105,)

'melspectrogram'

(7105, 60, 259)

'mfcc'

(7105, 20, 259)

'chroma'

(7105, 12, 259)

'rms'

(7105, 1, 259)

In [47]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(1896,)

'melspectrogram'

(1896, 60, 259)

'mfcc'

(1896, 20, 259)

'chroma'

(1896, 12, 259)

'rms'

(1896, 1, 259)

### Encode Classes

In [48]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [49]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

7105

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

1896

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [50]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [51]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           2.09090982e-14, 1.00035500e-07, 4.10201886e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           2.42591675e-14, 1.00793242e-07, 4.13385302e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.06456201e-14, 1.02097112e-07, 4.18868937e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.76671411e-08, 2.59462464e-08, 1.32848568e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           2.24950763e-10, 2.39690650e-09, 9.34205219e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           6.16144049e-13, 1.95562647e-09, 8.06973052e-08]],
  
         [[1.73426295e-05, 3.63408438e-06, 4.80290598e-13, ...,
           1.60876280e-14, 7.00188949e-08, 1.07775013e-06],
          [1.84753151e-05, 3.87795431e-06, 5.39064175e-13, ...,
           2.48021127e

### Save the merged dictionary to a pkl

In [52]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_6s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

## **Window Size = 4s**

### **['melspectrogram']**

In [53]:
features_list = ['melspectrogram']

In [54]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:13<00:00,  6.64it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:17<00:00,  7.27it/s]


In [55]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60, 173)

In [56]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [57]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60, 173)

### Encode Classes

In [58]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [59]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [60]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [61]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           1.06072697e-14, 6.16832790e-08, 5.78963081e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.12205350e-14, 6.15759939e-08, 5.77909589e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           1.25414211e-14, 6.13728398e-08, 5.75930686e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           1.16200449e-08, 1.17388327e-08, 3.10218327e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           1.11483545e-10, 1.11642392e-09, 9.85757182e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           1.07090598e-13, 3.56324655e-10, 3.34282947e-08]],
  
         [[1.22058890e-04, 9.34677081e-06, 3.22876749e-15, ...,
           9.62427203e-15, 2.13093339e-07, 5.59486302e-06],
          [1.24266710e-04, 9.51594868e-06, 1.18513276e-14, ...,
           1.10033380e

### Save the merged dictionary to a pkl

In [62]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_4s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [63]:
features_list = ['melspectrogram', 'mfcc']

In [64]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [03:02<00:00,  2.65it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:43<00:00,  2.98it/s]


In [65]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60, 173)

'mfcc'

(10981, 20, 173)

In [66]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60, 173)

'mfcc'

(2928, 20, 173)

### Encode Classes

In [67]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [68]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [69]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [70]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           1.06072697e-14, 6.16832790e-08, 5.78963081e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.12205350e-14, 6.15759939e-08, 5.77909589e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           1.25414211e-14, 6.13728398e-08, 5.75930686e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           1.16200449e-08, 1.17388327e-08, 3.10218327e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           1.11483545e-10, 1.11642392e-09, 9.85757182e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           1.07090598e-13, 3.56324655e-10, 3.34282947e-08]],
  
         [[1.22058890e-04, 9.34677081e-06, 3.22876749e-15, ...,
           9.62427203e-15, 2.13093339e-07, 5.59486302e-06],
          [1.24266710e-04, 9.51594868e-06, 1.18513276e-14, ...,
           1.10033380e

### Save the merged dictionary to a pkl

In [71]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_4s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [72]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [73]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [04:29<00:00,  1.80it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:12<00:00,  1.80it/s]


In [74]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60, 173)

'mfcc'

(10981, 20, 173)

'chroma'

(10981, 12, 173)

In [75]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60, 173)

'mfcc'

(2928, 20, 173)

'chroma'

(2928, 12, 173)

### Encode Classes

In [76]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [77]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [78]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [79]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           1.06072697e-14, 6.16832790e-08, 5.78963081e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.12205350e-14, 6.15759939e-08, 5.77909589e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           1.25414211e-14, 6.13728398e-08, 5.75930686e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           1.16200449e-08, 1.17388327e-08, 3.10218327e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           1.11483545e-10, 1.11642392e-09, 9.85757182e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           1.07090598e-13, 3.56324655e-10, 3.34282947e-08]],
  
         [[1.22058890e-04, 9.34677081e-06, 3.22876749e-15, ...,
           9.62427203e-15, 2.13093339e-07, 5.59486302e-06],
          [1.24266710e-04, 9.51594868e-06, 1.18513276e-14, ...,
           1.10033380e

### Save the merged dictionary to a pkl

In [80]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_4s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [81]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [82]:
features = Extraction(train_data,
                      val_data,
                      window_size=4,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [05:00<00:00,  1.61it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:22<00:00,  1.57it/s]


In [83]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(10981,)

'melspectrogram'

(10981, 60, 173)

'mfcc'

(10981, 20, 173)

'chroma'

(10981, 12, 173)

'rms'

(10981, 1, 173)

In [84]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(2928,)

'melspectrogram'

(2928, 60, 173)

'mfcc'

(2928, 20, 173)

'chroma'

(2928, 12, 173)

'rms'

(2928, 1, 173)

### Encode Classes

In [85]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [86]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

10981

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

2928

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [87]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [88]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           1.06072697e-14, 6.16832790e-08, 5.78963081e-06],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.12205350e-14, 6.15759939e-08, 5.77909589e-06],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           1.25414211e-14, 6.13728398e-08, 5.75930686e-06],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           1.16200449e-08, 1.17388327e-08, 3.10218327e-07],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           1.11483545e-10, 1.11642392e-09, 9.85757182e-08],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           1.07090598e-13, 3.56324655e-10, 3.34282947e-08]],
  
         [[1.22058890e-04, 9.34677081e-06, 3.22876749e-15, ...,
           9.62427203e-15, 2.13093339e-07, 5.59486302e-06],
          [1.24266710e-04, 9.51594868e-06, 1.18513276e-14, ...,
           1.10033380e

### Save the merged dictionary to a pkl

In [89]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_4s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

## **Window Size = 2s**

### **['melspectrogram']**

In [90]:
features_list = ['melspectrogram']

In [91]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [01:32<00:00,  5.27it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:22<00:00,  5.79it/s]


In [92]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60, 87)

In [93]:
train_y

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', ..., 'Anas strepera', 'Anas strepera',
       'Anas strepera'], dtype='<U25')

In [94]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60, 87)

### Encode Classes

In [95]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [96]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [97]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [98]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           5.40252245e-16, 1.56153176e-05, 1.55124408e-04],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.44704444e-15, 1.58973962e-05, 1.57929313e-04],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.38392743e-15, 1.63767277e-05, 1.62701248e-04],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.08672234e-08, 1.99395944e-05, 1.89381415e-04],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           4.94281655e-10, 1.80252409e-05, 1.73290591e-04],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           2.21867693e-13, 1.70996799e-05, 1.64420217e-04]],
  
         [[2.21674682e-07, 8.28132378e-09, 1.14113669e-16, ...,
           5.64442031e-13, 4.15756946e-08, 3.75276195e-07],
          [2.63981337e-07, 9.74223593e-09, 2.37362292e-16, ...,
           5.92175932e

### Save the merged dictionary to a pkl

In [99]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_2s_mel.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc']**

In [100]:
features_list = ['melspectrogram', 'mfcc']

In [101]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [03:18<00:00,  2.44it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [00:48<00:00,  2.69it/s]


In [102]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60, 87)

'mfcc'

(22631, 20, 87)

In [103]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60, 87)

'mfcc'

(6044, 20, 87)

### Encode Classes

In [104]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [105]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [106]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [107]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           5.40252245e-16, 1.56153176e-05, 1.55124408e-04],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.44704444e-15, 1.58973962e-05, 1.57929313e-04],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.38392743e-15, 1.63767277e-05, 1.62701248e-04],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.08672234e-08, 1.99395944e-05, 1.89381415e-04],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           4.94281655e-10, 1.80252409e-05, 1.73290591e-04],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           2.21867693e-13, 1.70996799e-05, 1.64420217e-04]],
  
         [[2.21674682e-07, 8.28132378e-09, 1.14113669e-16, ...,
           5.64442031e-13, 4.15756946e-08, 3.75276195e-07],
          [2.63981337e-07, 9.74223593e-09, 2.37362292e-16, ...,
           5.92175932e

### Save the merged dictionary to a pkl

In [108]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_2s_mel_mfcc.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma']**

In [109]:
features_list = ['melspectrogram', 'mfcc', 'chroma']

In [110]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [06:28<00:00,  1.25it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:26<00:00,  1.50it/s]


In [111]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60, 87)

'mfcc'

(22631, 20, 87)

'chroma'

(22631, 12, 87)

In [112]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60, 87)

'mfcc'

(6044, 20, 87)

'chroma'

(6044, 12, 87)

### Encode Classes

In [113]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [114]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [115]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [116]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           5.40252245e-16, 1.56153176e-05, 1.55124408e-04],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.44704444e-15, 1.58973962e-05, 1.57929313e-04],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.38392743e-15, 1.63767277e-05, 1.62701248e-04],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.08672234e-08, 1.99395944e-05, 1.89381415e-04],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           4.94281655e-10, 1.80252409e-05, 1.73290591e-04],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           2.21867693e-13, 1.70996799e-05, 1.64420217e-04]],
  
         [[2.21674682e-07, 8.28132378e-09, 1.14113669e-16, ...,
           5.64442031e-13, 4.15756946e-08, 3.75276195e-07],
          [2.63981337e-07, 9.74223593e-09, 2.37362292e-16, ...,
           5.92175932e

### Save the merged dictionary to a pkl

In [117]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_2s_mel_mfcc_chroma.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)

### **['melspectrogram', 'mfcc', 'chroma', 'rms']**

In [118]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'rms']

In [119]:
features = Extraction(train_data,
                      val_data,
                      window_size=2,
                      features=features_list,
                      avgpool=False
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [07:37<00:00,  1.06it/s]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:38<00:00,  1.32it/s]


In [120]:
train_y = features.train_y
display(train_y.shape)

train_features = features.train_features
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)

(22631,)

'melspectrogram'

(22631, 60, 87)

'mfcc'

(22631, 20, 87)

'chroma'

(22631, 12, 87)

'rms'

(22631, 1, 87)

In [121]:
val_y = features.val_y
display(val_y.shape)

val_features = features.val_features
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)

(6044,)

'melspectrogram'

(6044, 60, 87)

'mfcc'

(6044, 20, 87)

'chroma'

(6044, 12, 87)

'rms'

(6044, 1, 87)

### Encode Classes

In [122]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [123]:
display(len(train_y))
display(train_y[:10])

display(len(val_y))
display(val_y[:10])

22631

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

6044

array(['Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon',
       'Acrocephalus melanopogon', 'Acrocephalus melanopogon'],
      dtype='<U25')

In [124]:
train_features['label'] = train_y_encoded
val_features['label'] = val_y_encoded

In [125]:
merged_dict = {'train': train_features, 'val': val_features}
merged_dict

{'train': {'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
           5.40252245e-16, 1.56153176e-05, 1.55124408e-04],
          [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
           1.44704444e-15, 1.58973962e-05, 1.57929313e-04],
          [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
           3.38392743e-15, 1.63767277e-05, 1.62701248e-04],
          ...,
          [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
           4.08672234e-08, 1.99395944e-05, 1.89381415e-04],
          [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
           4.94281655e-10, 1.80252409e-05, 1.73290591e-04],
          [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
           2.21867693e-13, 1.70996799e-05, 1.64420217e-04]],
  
         [[2.21674682e-07, 8.28132378e-09, 1.14113669e-16, ...,
           5.64442031e-13, 4.15756946e-08, 3.75276195e-07],
          [2.63981337e-07, 9.74223593e-09, 2.37362292e-16, ...,
           5.92175932e

### Save the merged dictionary to a pkl

In [126]:
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_2s_all.pkl', 'wb') as file:
  pickle.dump(merged_dict, file)